# Pipeline config files

PyOPIA makes use of TOML for configuration files, which can used loaded and given to {class}`pyopia.pipeline.Pipeline`, or passed directly from command line, like this: `pyopia process config.toml`` ('Commmand line tools' page).

The below shows a typical example for a SilCam processing pipeline. This setup could be used for other standard images by adapting the load function.

```
[general] # general settings independent of processing steps
raw_files = 'raw_data/*.silc' # string used to obtain list of raw data files for processing
pixel_size = 24 # pixel size of imaging system in microns

[steps] # setup of analysis pipeline order, functions, and parameters

    [steps.classifier]
    pipeline_class = 'pyopia.classify.Classify'
    model_path = 'keras_model.h5' # path to trained nn model

    [steps.createbackground]
    pipeline_class = 'pyopia.background.CreateBackground'
    average_window = 10 # number of images used to create background
    instrument_module = 'silcam' # used to determina method for loading images. Either 'silcam', 'holo', or 'imread'

    [steps.load]
    pipeline_class = 'pyopia.instrument.silcam.SilCamLoad'

    [steps.correctbackground]
    pipeline_class = 'pyopia.background.CorrectBackgroundAccurate'
    bgshift_function = 'accurate' # optional 'fast' or 'accurate' method for moving backgrounds. For static background use 'pass' or comment this line.

    [steps.imageprep]
    pipeline_class = 'pyopia.instrument.silcam.ImagePrep'
    image_level = 'imraw' # the level of processing for further analysis. Either 'imraw' for ignoring background or 'imc' for using the backgroun-corrected image. Defaults to 'imc' if not defined.

    [steps.segmentation]
    pipeline_class = 'pyopia.process.Segment'
    threshold = 0.85 # threshold used for segmentation

    [steps.statextract]
    pipeline_class = 'pyopia.process.CalculateStats'

    [steps.output]
    pipeline_class = 'pyopia.io.StatsToDisc'
    output_datafile = 'proc/test' # prefix path for output nc file

```

Here is a typical configuration of a holographic reconstruction pipeline.

```
[general] # general settings independent of processing steps
raw_files = 'holo_test_data_01/*.pgm' # string used to obtain list of raw data files for processing
pixel_size = 4.4 # pixel size of imaging system in microns'wavelength = 658, # laser wavelength in nm

[steps] # setup of analysis pipeline order, functions, and parameters
    [steps.initial]
    pipeline_class = 'pyopia.instrument.holo.Initial'
    # hologram reconstruction settings
    wavelength = 658 # laser wavelength in nm
    n = 1.33 # index of refraction of sample volume medium (1.33 for water)
    offset = 27 # offset to start of sample volume in mm
    minZ = 0 # minimum reconstruction distance within sample volume in mm
    maxZ = 50 # maximum reconstruction distance within sample volume in mm
    stepZ = 0.5 #step size in mm

    [steps.classifier]
    pipeline_class = 'pyopia.classify.Classify'
    model_path = 'keras_model.h5' # path to trained nn model

    [steps.createbackground]
    pipeline_class = 'pyopia.background.CreateBackground'
    average_window = 10 # number of images used to create background
    instrument_module = 'holo' # used to determina method for loading images. Either 'silcam', 'holo', or 'imread'

    [steps.load]
    pipeline_class = 'pyopia.instrument.holo.Load'

    [steps.correctbackground]
    pipeline_class = 'pyopia.background.CorrectBackgroundAccurate'
    bgshift_function = 'accurate' # optional 'fast' or 'accurate' method for moving backgrounds. For static background use 'pass' or comment this line.

    [steps.reconstruct]
    pipeline_class = 'pyopia.instrument.holo.Reconstruct'
    stack_clean = 0.02

    [steps.focus]
    pipeline_class = 'pyopia.instrument.holo.Focus'
    stacksummary_function = 'max_map'
    threshold = 0.9
    increase_depth_of_field = true
    focus_function = 'find_focus_sobel'
    merge_adjacent_particles = 2

    [steps.segmentation]
    pipeline_class = 'pyopia.process.Segment'
    threshold = 0.9 # threshold used for segmentation

    [steps.statextract]
    pipeline_class = 'pyopia.process.CalculateStats'
    propnames = ['major_axis_length', 'minor_axis_length', 'equivalent_diameter', 
                              'feret_diameter_max', 'equivalent_diameter_area']

    [steps.output]
    pipeline_class = 'pyopia.io.StatsToDisc'
    output_datafile = 'proc/test' # prefix path for output nc file
```

## Exporting ROIs

If particle ROIs(regions of interest) are exported, the names of the ROIs are given by export_name in the STATS data, which includes their unique particle number ID as part of the name. This can be used to make nice looking summary montages, for example, using {func}`pyopia.statistics.make_montage` and {func}`pyopia.plotting.montage_plot`. To do, this, you need to specify a folder with 'export_outputpath' in the {class}´pyopia.process.CalculateStats`` step:

```
[steps.statextract]
    pipeline_class = 'pyopia.process.CalculateStats'
    export_outputpath = 'exported_rois'
```

If you don't want to export ROIs, then just leave our the 'export_outputpath' option, or set it to 'None'.